In [2]:
import torch

In [3]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
    [0.55, 0.87, 0.66],  # journey
    [0.57, 0.85, 0.64],  # starts
    [0.22, 0.58, 0.33],  # with
    [0.77, 0.25, 0.10],  # one
    [0.05, 0.80, 0.55]]  # step
)

In [4]:
x_2 = inputs[1]
d_in = inputs.shape[1] # tamanho do input embedding, d = 3
d_out = 2 # tamanho do output embedding d_out = 2

In [5]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [6]:
# x_2 é journey, então vamos encontrar a query para ele
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [7]:
keys = inputs @ W_key
values = inputs @ W_value
queries = inputs @ W_query

print("keys.shape: ", keys.shape)
print("values.shape: ", values.shape)
print("queries.shape: ", queries.shape)

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])
queries.shape:  torch.Size([6, 2])


In [8]:
keys_2 = keys[1] # keys para journey
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.6307)


In [9]:
# query_2 é de dimensão 1x2
attn_score_2 = query_2 @ keys.T
print(attn_score_2)

tensor([1.3621, 1.6307, 1.5975, 0.9023, 0.5511, 1.2828])


In [10]:
# Queries 6x2 e keys 2x6, resulta em matriz 6x6
attn_scores = queries @ keys.T # Produto escalar entre vetores
print(attn_scores) # cada linha simboliza os attn scores entre as queries e todas as outras chaves

tensor([[1.0092, 1.1920, 1.1677, 0.6576, 0.4014, 0.9366],
        [1.3621, 1.6307, 1.5975, 0.9023, 0.5511, 1.2828],
        [1.3437, 1.6096, 1.5769, 0.8908, 0.5441, 1.2663],
        [0.7477, 0.8950, 0.8768, 0.4952, 0.3025, 0.7041],
        [0.6339, 0.7769, 0.7612, 0.4320, 0.2642, 0.6123],
        [0.9758, 1.1587, 1.1351, 0.6400, 0.3907, 0.9109]])


In [11]:
# Normalização para obter attention weights
# Escalar valores para sqrt(keys_dim) antes de softmax
# No caso, keys_dim = 2
keys_dim = keys.shape[-1]
attn_weights = torch.softmax(attn_scores / keys_dim**0.5, dim=-1)
print(attn_weights)

tensor([[0.1774, 0.2019, 0.1984, 0.1384, 0.1154, 0.1685],
        [0.1779, 0.2151, 0.2101, 0.1285, 0.1002, 0.1682],
        [0.1777, 0.2145, 0.2096, 0.1290, 0.1010, 0.1683],
        [0.1742, 0.1933, 0.1908, 0.1457, 0.1271, 0.1689],
        [0.1718, 0.1900, 0.1879, 0.1489, 0.1322, 0.1692],
        [0.1766, 0.2010, 0.1977, 0.1393, 0.1168, 0.1687]])


In [12]:
context_vector = attn_weights @ values
print(context_vector)

tensor([[0.3507, 0.8808],
        [0.3566, 0.8973],
        [0.3563, 0.8966],
        [0.3464, 0.8692],
        [0.3446, 0.8644],
        [0.3502, 0.8795]])


In [13]:
class SelfAttention_v1(torch.nn.Module):
    
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))
    
    # Computa keys values e queries
    def forward(self, x):
        keys = x @ self.W_key
        values = x @ self.W_value
        queries = x @ self.W_query
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        
        context_vector = attn_weights @ values
        return context_vector

In [14]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs)) # Chama a função de forward

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [15]:
# Utilizando torch.nn.Linear para inicializar matrizes de pesos
# de forma computacionalmente mais eficiente
class SelfAttention_v2(torch.nn.Module):
    
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
    
    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        context_vector = attn_weights @ values
        return context_vector

In [16]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [17]:
# Mascarando attention weights para causal attention
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [18]:
context_length = attn_scores.shape[0] # quantas palavras o modelo pode olhar antes de prever a próxima
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [19]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [20]:
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums   # Causa data leakage - tokens futuros já afetaram
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [21]:
# Melhor jeito para não causar data leakage é colocar tokens futuros como -inf
# e após isso aplicar a função softmax para realizar a normalização
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf) # verifica onde tem 1 na matriz triu e substitui por -inf
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [22]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [23]:
context_vector = attn_weights @ values
print(context_vector)

tensor([[0.3669, 0.7646],
        [0.4012, 0.9337],
        [0.4120, 0.9904],
        [0.3735, 0.9221],
        [0.3357, 0.8055],
        [0.3350, 0.8293]], grad_fn=<MmBackward0>)


In [24]:
# Dropout -> previne overfitting e melhora generalização
# Pode ser utilizado após calcular attention scores e depois de obter context vector
# Desligar (zerar) attention scores aleatoriamente
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # Desliga em média 50% dos pesos
example = torch.ones(6, 6)
print(dropout(example)) # Para compensar pelo desligamento, os elementos restantes são escalados em 1 / (1 - p), sendo p = 0.5

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [25]:
torch.manual_seed(123)
print(attn_weights)
print(dropout(attn_weights))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [26]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 batches 6x3

torch.Size([2, 6, 3])


In [27]:
class CausalAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        # usamos buffer para mover automaticamente para cpu ou gpu junto com o modelo
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # criando máscara
    
    def forward(self, x):
        batch_dim, num_tokens, d_in = x.shape # batch é o número de batches
        keys = self.W_key(x) # multiplica input pela matriz de pesos keys
        values = self.W_value(x)
        queries = self.W_query(x)
        
        # batches processados sequencialmente
        # matriz queries vira lista de batches
        attn_scores = queries @ keys.transpose(1, 2) # transpões apenas dimensões num_tokens e dimensões de input para possibilitar multiplicação
        # :num_tokens para levar em conta casos que o número de tokens no batch é menor do que o context_length suportado
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context_vector = attn_weights @ values
        return context_vector
        

In [28]:
torch.manual_seed(123)
context_length = batch.shape[1] # pois temos um batch size de 6
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vectors = ca(batch)
print(context_vectors.shape) # 2x6x2 pois temos 2 inputs e cada context vector tem tamanho 6x2

torch.Size([2, 6, 2])


In [29]:
class MultiHeadAttentionWrapper(torch.nn.Module):
    # d_out é a dimensão do output que queremos
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
        )
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [30]:
torch.manual_seed(123)
context_length = batch.shape[1] # número de tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vectors = mha(batch)
print(context_vectors)
print("context_vectors.shape: ", context_vectors.shape) # 4 dimensões pois possui 2 heads e d_out = 2, então concatenamos

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vectors.shape:  torch.Size([2, 6, 4])


In [31]:
# Combinando classes anteriores
class MultiHeadAttention(torch.nn.Module):
    
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out deve ser divisível por num_heads"
    
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out) # camada linear para combinar outputs de cada head
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        
        # dividindo matrizes implicitamente ao adicionar dimensão num_heads
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        
        # transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # agrupando por head, cada uma vai ter uma matriz de attention score
        # inicialmente agrupava por número de tokens, mas não é eficiente para calcularmos attention scores para cada cabeça paralelamente
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        # computando attention scores
        # após a multiplicação, fica com dimensões (b, num_heads, num_tokens, num_tokens), e simboliza o valor de atenção que precisa dar p cada palavra
        attn_scores = queries @ keys.transpose(2, 3) # o que é importante é num_tokens e head_dim para calcularmos produto escalar
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # attention weights tem dimensões (b, num_heads, num_tokens, num_tokens)
        # values tem dimensões (b, num_heads, num_tokens, head_dim)
        # Attention weights e values tem a mesma dimensão como linhas, então é possível multiplicar
        # dimensão final: (b, num_heads, num_tokens, head_dim)
        # com isso, transpomos a matriz para termos (b, num_tokens, num_heads, head_dim) para podermos dar merge no num_heads e head_dim
        context_vector = (attn_weights @ values).transpose(1, 2)
        
        # combinando heads, self_d_out = self.num_heads * self_head_dim
        # Fica com dimensão (b, num_tokens, d_out)
        # contiguous() serve para colocar as matrizes no mesmo bloco de memória para fazermos o merge
        context_vector = context_vector.contiguous().view(b, num_tokens, self.d_out)
        context_vector = self.out_proj(context_vector) # projeção opcional
        
        return context_vector

In [32]:
# Testando multi head attention
torch.manual_seed(123)

inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vectors = mha(batch)
print(context_vectors)
print("context_vectors.shape: ", context_vectors.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>)
context_vectors.shape:  torch.Size([2, 3, 6])
